<a href="https://colab.research.google.com/github/iuryrosal/deep-learning/blob/master/Desafio_PrevisaoDeNotasENEM2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto - Previsão das Notas de Matemática dos participantes do ENEM 2016
Esse projeto envolveu um processo seletivo para um programa de aceleração de desenvolvimento ofertado pela codenation em 2020 na área de ciência de dados. 

**Descrição do Desafio:**

Você deverá criar um modelo para prever a nota da prova de matemática de quem participou do ENEM 2016. O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o Dicionário dos Microdados do Enem 2016.

Muitas universidades brasileiras utilizam o ENEM para selecionar seus futuros alunos e alunas. Isto é feito com uma média ponderada das notas das provas de matemática, ciências da natureza, linguagens e códigos, ciências humanas e redação, com os pesos abaixo:

matemática: 3
ciências da natureza: 2
linguagens e códigos: 1.5
ciências humanas: 1
redação: 3

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna NU_NOTA_MT) de quem participou do ENEM 2016. Salve sua resposta em um arquivo chamado answer.csv com duas colunas: NU_INSCRICAO e NU_NOTA_MT.



## Planejamento inicial
Para esse desafio, decidi utilizar o Python envolvendo o conceito de redes neurais para regressão. Inicialmente precisaria importar os arquivos disponibilizados e avaliar como deveria trabalhar. Ao longo do desafio montei diferentes estruturas e fiz várias modificações até chegar no melhor resultado possível. Por motivos didáticos, irei já apresentar o modelo final que garantiu um acerto de 93,72% na previsão das notas.

## Compreendendo os arquivos 

In [2]:
from google.colab import files
import numpy as np
import pandas as pd
import io
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split #biblioteca para machine learning
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [3]:
#Carregamento do Arquivo teste.csv
uploaded = files.upload()
uploaded
access = uploaded['test.csv']
file_str = io.StringIO(access.decode('latin-1'))
test = pd.read_csv(file_str, encoding='latin-1')
test.head(8)

Saving test.csv to test.csv


,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01abbb7f1a90505385f44eec9905f82ca2a42cfd,81d0ee00ef42a7c23eb04496458c03d4c5b9c31a,464.8,443.5,431.8,0,1.0,120.0,80.0,80.0,100.0,40.0,420.0,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,c8328ebc6f3238e06076c481bc1b82b8301e7a3f,f48d390ab6a2428e659c37fb8a9d00afde621889,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,391.1,491.1,548.0,1,1.0,120.0,120.0,120.0,120.0,100.0,580.0,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,3.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,595.9,622.7,613.6,0,1.0,80.0,40.0,40.0,80.0,80.0,320.0,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,66b1dad288e13be0992bae01e81f71eca1c6e8a6,942ab3dc020af4cf53740b6b07e9dd7060b24164,5aebe5cad7fabc1545ac7fba07a4e6177f98483c,767a32545304ed293242d528f54d4edb1369f910,592.9,492.6,571.4,1,1.0,100.0,80.0,60.0,80.0,0.0,320.0,D,H,H,C,B,A,NaN,A
5,e656d6bad65c93fb2880f1eba5037008c8e75774,43,RS,18,M,3,1,1,1,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,909237ab0d84688e10c0470e2997348aff585273,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,414.5,568.5,542.5,1,1.0,120.0,120.0,120.0,120.0,80.0,560.0,E,E,C,B,B,A,NaN,A
6,465cd2a6907fb37d9d8ad3c065f0e2dabdba9b13,35,SP,17,F,1,1,2,0,3,1.0,0,4.0,0,0,0,0,0,0,0,0,1,1,1,66b1dad288e13be0992bae01e81f71eca1c6e8a6,942ab3dc020af4cf53740b6b07e9dd7060b24164,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,485.7,654.6,606.6,0,1.0,140.0,120.0,120.0,120.0,120.0,620.0,D,E,H,D,B,A,NaN,D
7,11539e86171bf07d3a36f09377d7f54ebcc8406a,23,CE,17,F,3,2,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,909237ab0d84688e10c0470e2997348aff585273,01abbb7f1a90505385f44eec9905f82ca2a42cfd,81d0ee00ef42a7c23eb04496458c03d4c5b9c31a,423.2,521.0,451.8,0,1.0,100.0,80.0,80.0,100.0,60.0,420.0,D,C,B,A,A,A,NaN,A


In [4]:
#Carregamento do Arquivo train.csv
uploaded = files.upload()
uploaded
access = uploaded['train.csv']
file_str = io.StringIO(access.decode('latin-1'))
train = pd.read_csv(file_str, encoding='latin-1')
train.head(8)

Saving train.csv to train.csv


,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,...,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,1,4314902.0,Porto Alegre,43.0,RS,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A,C,H,E,5.0,5.0,1.0,0.0,0.0,5,5,0,5,5,5,5,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,1,2304707.0,Granja,23.0,CE,2,0,2,1.0,0,23005157.0,2304707.0,Granja,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,1,1,1,1,1,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,1,2304400.0,Fortaleza,23.0,CE,3,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,5,5,5,5,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,1,3304557.0,Rio de Janeiro,33.0,RJ,1,9,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,B,B,A,B,A,B,C,A,B,D,B,C,B,C,F,C,2.0,0.0,5.0,5.0,5.0,5,5,5,5,2,5,5,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,1,1505304.0,OriximinÃ¡,15.0,PA,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,A,A,A,B,A,A,B,A,A,A,B,F,D,5.0,4.0,3.0,5.0,4.0,5,5,4,3,1,4,5,NaN,A,A,A,A,A,A,A,A,A
5,6,a37c99ec251d4f6e8ddbeabadf1c87fdbfddc4d1,2016,2902005,Aracatu,29,BA,18,F,0.0,1,1,2933307.0,VitÃ³ria da Conquista,29.0,BA,1,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,B,B,B,A,A,B,A,A,B,B,B,E,A,B,B,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,5,5,5,5,5,5,NaN,A,A,A,A,A,A,A,A,A
6,7,63b4e128e9ffe8ab27f5d093db1976ef4f353e0a,2016,5102637,Campo Novo do Parecis,51,MT,18,F,0.0,3,1,2804508.0,Nossa Senhora da GlÃ³ria,28.0,SE,2,0,2,1.0,0,51010224.0,5102637.0,Campo Novo do Parecis,51.0,MT,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,B,A,B,A,A,A,D,A,B,B,C,E,B,2.0,0.0,2.0,4.0,2.0,5,5,2,5,2,0,2,0.0,A,A,A,B,B,A,A,A,A
7,8,2eb189d4912f64b19d1967e8e84b6141aba18770,2016,4301602,BagÃ©,43,RS,18,F,0.0,1,1,3550308.0,SÃ£o Paulo,35.0,SP,1,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,B,B,A,B,A,B,C,B,B,D,B,B,B,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,5,1,5,5,NaN,C,A,A,A,A,D,A,A,A


O arquivo train será utilizado para realizar o treinamento da rede neural como também alguns testes que auxiliaram nesse treinamento. Isso pois neste arquivos além de termos muitos parâmetros (cerca de 167), já que são as respostas do formulário de inscrição juntamente com os resultados da prova, já temos as notas de matemática, então conseguimos saber qual a melhor configuração a partir da comparação do que a rede neural está prevendo e as respostas corretas. O arquivo test possui apenas alguns parametros e não possui a nota de matemática incluida, justamente por que devemos prever a nota a partir dos parâmetros fornecidos. Utilizaremos o test apenas no final quando a rede neural estiver pronta na lua melhor configuração.

# Limpando colunas

Como deu para observar, o arquivo test possui bem menos colunas do que o train, isso ja é um guia de que algumas colunas não precisaremos utilizar para construir a rede neural. Por exemplo, no train existe as respostas de todas as questões do formulário de inscrição, no entanto, no test apenas temos de algumas, logo essas que estão no train mas não estão no test serão inutilizadas na hora da previsão das notas pelos inscritos na tabela test.

Outra observação é que alguns dados como códigos de inscrição, de provas, etc.. não são tal úteis pois envolvem apenas uma caracteristica em específico do candidato e não alguma informação que influenciará na nota.

Com esses dois aspectos conseguimos eliminar muitos parâmetros e assim trabalhar de uma maneira mais produtiva.

In [125]:
unecessary_columns = [i for i in train.columns if i not in test.columns]
unecessary_columns
new_train = train.drop(columns=unecessary_columns)
new_train['NU_NOTA_MT'] = train['NU_NOTA_MT'] #adicionando a coluna de notas
new_train.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,NU_NOTA_MT
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,43,RS,24,M,1,1,1,4,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,436.3,495.4,581.2,1,1.0,120.0,120.0,120.0,80.0,80.0,520.0,D,D,C,A,A,C,H,A,399.4
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,23,CE,17,F,3,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,909237ab0d84688e10c0470e2997348aff585273,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,474.5,544.1,599.0,1,1.0,140.0,120.0,120.0,120.0,80.0,580.0,A,A,B,A,A,A,NaN,A,459.8
2,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,23,CE,21,F,3,1,3,0,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,D,C,A,A,A,NaN,A,NaN
3,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,33,RJ,25,F,0,1,1,9,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,E,E,C,B,C,F,D,NaN
4,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,13,AM,28,M,2,1,1,4,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,D,C,A,A,B,F,A,NaN


In [0]:
train_clean = new_train.drop(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5'], axis=1)


In [127]:
train_clean

,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047,NU_NOTA_MT
0,24,M,1,1,1,4,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,436.3,495.4,581.2,520.0,D,D,C,A,A,C,H,A,399.4
1,17,F,3,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,474.5,544.1,599.0,580.0,A,A,B,A,A,A,NaN,A,459.8
2,21,F,3,1,3,0,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,D,D,C,A,A,A,NaN,A,NaN
3,25,F,0,1,1,9,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,H,E,E,C,B,C,F,D,NaN
4,28,M,2,1,1,4,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,E,D,C,A,A,B,F,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,20,M,1,1,1,3,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,438.2,419.7,394.5,440.0,E,E,G,B,B,B,C,A,403.2
13726,33,F,3,2,1,10,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,423.8,557.9,533.0,560.0,A,A,C,A,B,B,I,A,452.4
13727,15,F,1,1,3,0,1,NaN,1,NaN,0,0,0,0,0,0,0,0,1,1,1,460.5,528.9,569.3,600.0,C,G,B,A,B,A,NaN,A,398.0
13728,36,M,3,1,4,0,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,422.5,621.7,569.0,460.0,B,D,G,B,B,C,E,A,386.6


De 167 parâmetros, agora temos apenas 34 parâmetros.
### Descrição dos parâmetros
**NU_IDADE** - Idade do candidato <br/>
**TP_SEXO** - Sexo do participante <br/>
**TP_COR_RACA** - Cor/raça (0- Não Declarado, 1- Branca, 2- Preta, 3- Parda, 4- Amarela, 5- Indígena, 6- Não dispõe da informação) <br/>
**TP_NACIONALIDADE** - Nacionalidade (0- Não Informado, 1- Brasileiro(a), 2- Brasileiro(a) Naturalizado(a), 3- Estrangeiro(a), 4- Brasileiro(a)Nato(a), nascido(a) no exterior). <br/>
**TP_ST_CONCLUSAO** - Situação de conclusão do Ensino Médio (1- Já concluiu o Ensino Médio, 2- Estou cursando e concluirei o Ensino Médio em 2016, 3-Estou cursando e concluirei o Ensino Médio após 2016, 4- Não concluí e não estou cursando o Ensino Médio). <br/>
**TP_ANO_CONCLUIU** - Ano de Conclusão do Ensino Médio (0- Não Informado, 1- 2015, 2- 2014, 3- 2013, 4- 2012, 5- 2011, 6- 2010, 7- 2009, 8- 2008, 9- 2007, 10- Anterior a 2007). <br/>
**TP_ESCOLA**	- Tipo de escola do Ensino Médio (1- Não Respondeu, 2- Pública, 3- Privada, 4- Exterior). <br/>
**TP_ENSINO**	- Tipo de instituição que concluiu ou concluirá o Ensino Médio (1- Ensino Regular, 2- Educação Especial - Modalidade Substitutiva, 3- Educação de Jovens e Adultos). <br/>
**IN_TREINEIRO** - Indica se o inscrito fez a prova com intuito de apenas treinar seus conhecimentos (1- Sim, 0- Não). <br/>
**TP_DEPENDENCIA_ADM_ESC** - Dependência administrativa (Escola) (1- Federal, 2- Estadual, 3- Municipal, 4- Privada). <br/>
**IN_BAIXA_VISAO** - Indicador de baixa visão (1- Sim, 2- Não) <br/>
**IN_CEGUEIRA** - Indicador de cegueira (1- Sim, 2- Não) <br/>
**IN_SURDEZ** - Indicador de surdez (1- Sim, 2- Não) <br/>
**IN_DISLEXIA** - Indicador de dislexia (1- Sim, 0- Não) <br/>
**IN_DISCALCULIA** - Indicador de discalculia (1- Sim, 0- Não). <br/>
**IN_SABATISTA** - Indicador de solicitação de prova aplicada em horário diferenciado para os guardadores dos sábados (1- Sim, 2- Não). <br/>
**IN_GESTANTE** - Indicador de gestante (1- Sim, 2- Não). <br/>
**IN_IDOSO** - Indicador de inscrito idoso (1- Sim 2- Não). <br/>
**TP_PRESENCA_CN** - Presença na prova objetiva de Ciências da Natureza (0- faltou à prova, 1- Presenta na prova, 2- Eliminado na prova). <br/>
**TP_PRESENCA_CH** - Presença na prova objetiva de Ciências Humanas (0- faltou à prova, 1- Presente na prova, 2- Eliminado na prova). <br/>
**TP_PRESENCA_LC** - Presença na prova objetiva de Linguagens e Códigos (0- faltou à prova, 1- presenta na prova, 2- Eliminado na prova). <br/>
**NU_NOTA_CN** - Nota da prova de Ciências da Natureza. <br/>
**NU_NOTA_CH** - Nota da prova de Ciências da Natureza. <br/>
**NU_NOTA_LC** - Nota da prova de Linguagens e Códigos. <br/>
**NU_NOTA_MT** - Nota da prova de Matemática. <br/>
**NU_NOTA_REDACAO**	- Nota da prova de redação. <br/>
**Q001** - Até que série seu pai, ou o homem responsável por você, estudou? (A- Nunca estudoU, B- Não completou a 4ª série/5º ano do Ensino Fundamental, C- Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental, D- Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio, E- Completou o Ensino Médio, mas não completou a Faculdade, F- Completou a Faculdade, mas não completou a Pós-graduação, G- Completou a Pós-graduação, H- Não sei). <br/>
**Q002** - Até que série sua mãe, ou a mulher responsável por você, estudou? (A-  Nunca estudou, B- Não completou a 4ª série/5º ano do Ensino Fundamental, C- Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental, D- Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio, E- Completou o Ensino Médio, mas não completou a Faculdade, F- Completou a Faculdade, mas não completou a Pós-graduação, G- Completou a Pós-graduação, H- Não sei). <br/>
**Q006** - Qual é a renda mensal de sua família? (Some a sua renda com a dos seus familiares.) (A- Nenhuma, B- Até RS 880,00, C- De RS 880,01 até RS 1.320,00, D- De RS 1.320,01 até RS 1.760,00, E- De RS 1.760,01 até RS 2.200,00, F- De RS 2.200,01 até RS 2.640,00, G- De RS 2.640,01 até RS 3.520,00, H- De RS 3.520,01 até RS 4.400,00, I- De RS 4.400,01 até RS 5.280,00, J - De RS 5.280,01 até RS 6.160,00, K- De RS 6.160,01 até RS 7.040,00, L- De RS 7.040,01 até RS 7.920,00, M- De RS 7.920,01 até RS 8.800,00, N- De RS 8.800,01 até RS 10.560,00, O - De RS 10.560,01 até RS 13.200,00,  P- De RS 13.200,01 até RS 17.600,00, Q- Mais de RS 17.600,00). <br/>
**Q025** - Na sua residência tem acesso à Internet? (A- Não, B- Sim) <br/>
**Q026** - Você exerce ou já exerceu atividade remunerada? (A- Não, nunca trabalhei, B- Sim, já trabalhei, mas não estou trabalhando, C- Sim, estou trabalhando). <br/>
**Q027** - Com que idade você começou a exercer uma atividade remunerada? (A- Menos de 14 anos, B- 14 anos, C- 15 anos, D- 16 anos, E- 17 anos, F- 18 anos, G- 19 anos, H- 20 anos, I- 21 anos, J- 22 anos, K - 23 anos, L- 24 anos, M- 25 anos ou mais). <br/>
**Q047** - Em que tipo de escola você frequentou o Ensino Médio? (A- Somente em escola pública, B- Parte em escola pública e parte em escola privada sem bolsa de estudo integral, C- Parte em escola pública e parte em escola privada com bolsa de estudo integral, D- Somente em escola privada sem bolsa de estudo integral, E- Somente em escola privada com bolsa de estudo integral).



# Tratando dados nulos

Ao fazer a verificação foi perceptível algumas colunas com dados nulos. No caso da TP_DEPENDENCIA_ADM_ESC e TP_ENSINO foram substítuidos pela média arredondada. Na questão 27, as pessoas que deixaram essa parte em branco foi pela resposta da questão 26. Por conta disso, os dados em branco da questão 27 foram substituídos pela letra N. Logo, esse N representa as pessoas que não responderam por terem dito que não trabalhavam na questão 26. Jpa as notas das provas, que, obviamente, ficaram ausentes pois o aluno faltou a prova. Nessa situação, faz sentido substituir por 0. Querendo ou não, a falta na avaliação pode ser vista como uma nota nula. Ao substituir esses valores ausentes pela média, o resultado não se demonstrou muito assertivo...  

In [128]:
#verificando colunas nulas
train_clean.isnull().sum().values 

array([   0,    0,    0,    0,    0,    0,    0, 9448,    0, 9448,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3389,
       3389, 3597, 3597,    0,    0,    0,    0,    0,    0, 7373,    0,
       3597])

In [129]:
train_clean['TP_DEPENDENCIA_ADM_ESC'].mean()

2.259691732835124

In [130]:
train_clean['TP_ENSINO'].mean()

1.1788883699205979

In [0]:
train_clean['TP_DEPENDENCIA_ADM_ESC'].fillna(value=2, inplace=True)
train_clean['TP_ENSINO'].fillna(value=1, inplace=True)
#N é para substituir o NaN proveniente das pessoas que não trabalham, ai não faz sentido elas responderem algo nas questões 27 e 28.
train_clean['Q027'].fillna(value='N', inplace=True)

In [0]:
train_clean['NU_NOTA_REDACAO'].fillna(value=0.00, inplace=True)
train_clean['NU_NOTA_CN'].fillna(value=0.00, inplace=True)
train_clean['NU_NOTA_CH'].fillna(value=0.00, inplace=True)
train_clean['NU_NOTA_LC'].fillna(value=0.00, inplace=True)
train_clean['NU_NOTA_MT'].fillna(value=0.00, inplace=True)

In [133]:
#confirmando que todos os dados estão preenchidos
train_clean.isnull().sum().values 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Agora que tratamos os dados nulos, chegou a hora de trabalhar a tipagem. Primeiramente os dados do tipo inteiros precisam ser convertidos para o tipo float.

In [134]:
train_clean.dtypes

NU_IDADE                    int64
TP_SEXO                    object
TP_COR_RACA                 int64
TP_NACIONALIDADE            int64
TP_ST_CONCLUSAO             int64
TP_ANO_CONCLUIU             int64
TP_ESCOLA                   int64
TP_ENSINO                 float64
IN_TREINEIRO                int64
TP_DEPENDENCIA_ADM_ESC    float64
IN_BAIXA_VISAO              int64
IN_CEGUEIRA                 int64
IN_SURDEZ                   int64
IN_DISLEXIA                 int64
IN_DISCALCULIA              int64
IN_SABATISTA                int64
IN_GESTANTE                 int64
IN_IDOSO                    int64
TP_PRESENCA_CN              int64
TP_PRESENCA_CH              int64
TP_PRESENCA_LC              int64
NU_NOTA_CN                float64
NU_NOTA_CH                float64
NU_NOTA_LC                float64
NU_NOTA_REDACAO           float64
Q001                       object
Q002                       object
Q006                       object
Q024                       object
Q025          

In [0]:
train_clean['NU_IDADE'] = train_clean['NU_IDADE'].astype('float64')
train_clean['TP_COR_RACA'] = train_clean['TP_COR_RACA'].astype('float64')
train_clean['TP_NACIONALIDADE'] = train_clean['TP_NACIONALIDADE'].astype('float64')
train_clean['TP_ST_CONCLUSAO'] = train_clean['TP_ST_CONCLUSAO'].astype('float64')
train_clean['TP_ANO_CONCLUIU'] = train_clean['TP_ANO_CONCLUIU'].astype('float64')
train_clean['TP_ESCOLA'] = train_clean['TP_ESCOLA'].astype('float64')
train_clean['IN_TREINEIRO'] = train_clean['IN_TREINEIRO'].astype('float64')
train_clean['IN_BAIXA_VISAO'] = train_clean['IN_BAIXA_VISAO'].astype('float64')
train_clean['IN_CEGUEIRA'] = train_clean['IN_CEGUEIRA'].astype('float64')
train_clean['IN_SURDEZ'] = train_clean['IN_SURDEZ'].astype('float64')
train_clean['IN_DISLEXIA'] = train_clean['IN_DISLEXIA'].astype('float64')
train_clean['IN_DISCALCULIA'] = train_clean['IN_DISCALCULIA'].astype('float64')
train_clean['IN_SABATISTA'] = train_clean['IN_SABATISTA'].astype('float64')
train_clean['IN_GESTANTE'] = train_clean['IN_GESTANTE'].astype('float64')
train_clean['IN_IDOSO'] = train_clean['IN_IDOSO'].astype('float64')
train_clean['TP_PRESENCA_CN'] = train_clean['TP_PRESENCA_CN'].astype('float64')
train_clean['TP_PRESENCA_CH'] = train_clean['TP_PRESENCA_CH'].astype('float64')
train_clean['TP_PRESENCA_LC'] = train_clean['TP_PRESENCA_LC'].astype('float64')


In [136]:
train_clean.dtypes

NU_IDADE                  float64
TP_SEXO                    object
TP_COR_RACA               float64
TP_NACIONALIDADE          float64
TP_ST_CONCLUSAO           float64
TP_ANO_CONCLUIU           float64
TP_ESCOLA                 float64
TP_ENSINO                 float64
IN_TREINEIRO              float64
TP_DEPENDENCIA_ADM_ESC    float64
IN_BAIXA_VISAO            float64
IN_CEGUEIRA               float64
IN_SURDEZ                 float64
IN_DISLEXIA               float64
IN_DISCALCULIA            float64
IN_SABATISTA              float64
IN_GESTANTE               float64
IN_IDOSO                  float64
TP_PRESENCA_CN            float64
TP_PRESENCA_CH            float64
TP_PRESENCA_LC            float64
NU_NOTA_CN                float64
NU_NOTA_CH                float64
NU_NOTA_LC                float64
NU_NOTA_REDACAO           float64
Q001                       object
Q002                       object
Q006                       object
Q024                       object
Q025          

Os dados do tipo object (string) serão tratados posteriormente, pois representam dados categóricos (aqueles que são letras representando uma certa categoria).

# Previsores, Treinamento e Teste


Separamos os dados que serão a base para chegarmos a previsão da nota.

In [0]:
# colocando todas as colunas exceto NU_NOTA_MT
predictors = train_clean.iloc[:, lambda train_clean: train_clean.columns != 'NU_NOTA_MT'].values

In [138]:
predictors

array([[24.0, 'M', 1.0, ..., 'C', 'H', 'A'],
       [17.0, 'F', 3.0, ..., 'A', 'N', 'A'],
       [21.0, 'F', 3.0, ..., 'A', 'N', 'A'],
       ...,
       [15.0, 'F', 1.0, ..., 'A', 'N', 'A'],
       [36.0, 'M', 3.0, ..., 'C', 'E', 'A'],
       [17.0, 'M', 1.0, ..., 'A', 'N', 'A']], dtype=object)

Aqui separamos o resultado para ser um guia no treinamento da rede neural e verificarmos o nível de assertividade.

In [0]:
grades = train_clean.loc[:, 'NU_NOTA_MT'].values

In [140]:
grades

array([399.4, 459.8,   0. , ..., 398. , 386.6, 428.9])

Agora iremos tratar os dados categóricos ('A', por exemplo). Para isso usaremos o LabelEnconder juntamente com o OneHotEnconder. Nesse processo, cada dado virará um conjunto de parametros numericos. Por exempl, no caso do sexo, o 'M' virará algo do tipo 0 1 e o 'F' 1 0. Ou seja, estamos convertendo o dado categórico para algo numerico para assim podermos usar esse parâmetro na rede neural.

In [141]:
predictors.shape

(13730, 33)

In [0]:
labelenconder = LabelEncoder()
predictors[:, 1] = labelenconder.fit_transform(predictors[:, 1])
predictors[:, -8] = labelenconder.fit_transform(predictors[:, -8])
predictors[:, -7] = labelenconder.fit_transform(predictors[:, -7])
predictors[:, -6] = labelenconder.fit_transform(predictors[:, -6])
predictors[:, -5] = labelenconder.fit_transform(predictors[:, -5])
predictors[:, -4] = labelenconder.fit_transform(predictors[:, -4])
predictors[:, -3] = labelenconder.fit_transform(predictors[:, -3])
predictors[:, -2] = labelenconder.fit_transform(predictors[:, -2])
predictors[:, -1] = labelenconder.fit_transform(predictors[:, -1])

#onehotenconder = OneHotEncoder(categories = [1,-24,-25,-26,-27,-28,-29,-30,-31])
#previsores = onehotenconder.fit_transform(previsores).toarray()

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1,-1,-2,-3,-4,-5,-6,-7,-8])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)

predictors = ct.fit_transform(predictors)

Observe que após a aplicação do processo descrito acima, a quantidade de colunas aumentou.

In [143]:
predictors.shape

(13730, 88)

In [144]:
predictors[0]

array([0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,
       1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 24.0,
       1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 436.3, 495.4, 581.2, 520.0],
      dtype=object)

Agora iremos usar o train_test_split para dividir o grupo de previsores e notas para 75% para o treinamento e 25% para o teste. Neste caso não utilizamos a validação cruzada. 

In [0]:
predictors_train, predictors_test, class_train, class_test = train_test_split(predictors, grades, test_size=0.25) #75% para treinamento e 25% para teste

# Rede Neural

Aqui montamos a rede neural com 88 neurônios iniciais (1 para cada coluna) com 6 camadas ocultas. Utilizamos a função de ativação 'relu' pois foi a que mostrou ter o melhor resultado. Na camada de saída (com 1 neurônio apenas, pois terá a nota na saída) terá a função de ativação do tipo linear justamente por se tratar de uma regressão. No fim, utilizamos o optimizer sendo do tipo ADAM e a função loss sendo o Mean Absolute Error. Por fim, aplicamos as variáveis com os dados de treinamento, tendo um batch size de tamanho 5 (relacionado com a atualização dos pesos) e um total de 200 épocas.

In [146]:
regressor = Sequential()
regressor.add(Dense(units= 45, activation= 'relu', input_dim= 88))
regressor.add(Dense(units= 23, activation= 'relu'))
regressor.add(Dense(units= 12, activation= 'relu'))
regressor.add(Dense(units= 23, activation= 'relu'))
regressor.add(Dense(units= 45, activation= 'relu'))
regressor.add(Dense(units= 88, activation= 'relu'))
regressor.add(Dense(units= 1, activation= 'linear'))
regressor.compile(optimizer= 'adam', loss= 'mean_absolute_error', metrics= ['mean_absolute_error'])
regressor.fit(predictors_train, class_train, batch_size= 5, epochs= 200)

Epoch 1/200
10297/10297 [==============================] - 3s 275us/step - loss: 51.7029 - mean_absolute_error: 51.7029
Epoch 2/200
10297/10297 [==============================] - 3s 264us/step - loss: 47.7150 - mean_absolute_error: 47.7151
Epoch 3/200
10297/10297 [==============================] - 3s 259us/step - loss: 47.0213 - mean_absolute_error: 47.0213
Epoch 4/200
10297/10297 [==============================] - 3s 258us/step - loss: 46.5257 - mean_absolute_error: 46.5256
Epoch 5/200
10297/10297 [==============================] - 3s 263us/step - loss: 46.4695 - mean_absolute_error: 46.4695
Epoch 6/200
10297/10297 [==============================] - 3s 261us/step - loss: 46.2389 - mean_absolute_error: 46.2388
Epoch 7/200
10297/10297 [==============================] - 3s 262us/step - loss: 45.6841 - mean_absolute_error: 45.6841
Epoch 8/200
10297/10297 [==============================] - 3s 262us/step - loss: 46.0617 - mean_absolute_error: 46.0616
Epoch 9/200
10297/10297 [===============

Realizando uma verificação com as variáveis de teste para termos uma noção de como está ocorrendo a previsão.

In [0]:
result = regressor.predict(predictors_test)

In [148]:
result[:20]

array([[ 4.9462350e+02],
       [ 4.3769437e+02],
       [ 5.1885541e+02],
       [ 4.8757648e+02],
       [ 6.2421027e+02],
       [ 5.2891412e+02],
       [-5.5717945e-02],
       [-5.5717945e-02],
       [ 5.5787872e+02],
       [ 4.2071490e+02],
       [ 4.5007217e+02],
       [ 4.3978561e+02],
       [ 4.9260413e+02],
       [ 5.5443701e+02],
       [ 4.0973093e+02],
       [ 4.6274045e+02],
       [-5.5717945e-02],
       [ 4.4059464e+02],
       [ 4.1815964e+02],
       [ 4.6119424e+02]], dtype=float32)

In [149]:
class_test[:20]

array([520.6, 419.5, 401.3, 477. , 741.1, 560.2,   0. ,   0. , 608.1,
       446.2, 344.7, 394.4, 487.1, 401. , 447.7, 429. ,   0. , 457. ,
       480.3, 411.7])

In [150]:
result.mean()

355.927

In [151]:
class_test.mean()

351.76769589280514

Por meio da média conseguimos notar uma diferença muito pequena do resultado previsto e do real. Um detalhe interessante é que em alguns testes, o resultado daria um pouco negativo para quando a nota real é 0 ou previa uma nota muito baixa positivamente..

#Preparando a entrega


In [152]:
test

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01abbb7f1a90505385f44eec9905f82ca2a42cfd,81d0ee00ef42a7c23eb04496458c03d4c5b9c31a,464.8,443.5,431.8,0,1.0,120.0,80.0,80.0,100.0,40.0,420.0,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,c8328ebc6f3238e06076c481bc1b82b8301e7a3f,f48d390ab6a2428e659c37fb8a9d00afde621889,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,391.1,491.1,548.0,1,1.0,120.0,120.0,120.0,120.0,100.0,580.0,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,3.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,595.9,622.7,613.6,0,1.0,80.0,40.0,40.0,80.0,80.0,320.0,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,2d22ac1d42e6187f09ee6c578df187a760123ccf,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,66b1dad288e13be0992bae01e81f71eca1c6e8a6,942ab3dc020af4cf53740b6b07e9dd7060b24164,5aebe5cad7fabc1545ac7fba07a4e6177f98483c,767a32545304ed293242d528f54d4edb1369f910,592.9,492.6,571.4,1,1.0,100.0,80.0,60.0,80.0,0.0,320.0,D,H,H,C,B,A,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,41,PR,17,F,1,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01abbb7f1a90505385f44eec9905f82ca2a42cfd,81d0ee00ef42a7c23eb04496458c03d4c5b9c31a,398.3,558.2,511.6,0,1.0,120.0,120.0,120.0,100.0,40.0,500.0,E,E,D,A,B,A,NaN,A
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,35,SP,17,M,2,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,427.6,579.7,471.1,1,1.0,100.0,100.0,100.0,120.0,100.0,520.0,C,C,C,A,A,A,NaN,A
4573,655fa6306720ff16e825903b5422a46608a77545,35,SP,19,M,1,1,1,2,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,909237ab0d84688e10c0470e2997348aff585273,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,639.2,643.8,604.9,0,1.0,160.0,140.0,120.0,140.0,80.0,640.0,D,F,D,B,B,A,NaN,D
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,52,GO,17,M,2,2,3,0,1,NaN,1,NaN,0,0,0,0,0,0,0,0,1,1,1,66b1dad288e13be0992bae01e81f71eca1c6e8a6,942ab3dc020af4cf53740b6b07e9dd7060b24164,72f80e4b3150c627c7ffc93cfe0fa13a9989b610,577f8968d95046f5eb5cc158608e12fa9ba34c85,427.1,467.9,540.2,1,1.0,140.0,80.0,80.0,140.0,80.0,520.0,C,E,C,A,A,A,NaN,A


Agora vamos trabalhar a tabela de test. Precisamos deixar os mesmos parâmetros utilizados da train na rede neural construída. Então, precisamos remover alguns parâmetros:

In [0]:
test_clean = test.drop(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 
                          'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5'], axis=1)

In [154]:
test_clean

,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DISLEXIA,IN_DISCALCULIA,IN_SABATISTA,IN_GESTANTE,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,22,F,3,1,1,5,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,464.8,443.5,431.8,420.0,B,A,C,A,A,C,C,A
1,26,F,3,1,1,8,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,391.1,491.1,548.0,580.0,E,B,C,B,B,B,F,A
2,21,M,1,1,2,0,2,3.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,595.9,622.7,613.6,320.0,E,E,D,B,B,A,NaN,A
3,27,F,3,1,1,8,1,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,H,E,G,B,B,A,NaN,A
4,18,M,1,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,592.9,492.6,571.4,320.0,D,H,H,C,B,A,NaN,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,17,F,1,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,398.3,558.2,511.6,500.0,E,E,D,A,B,A,NaN,A
4572,17,M,2,1,2,0,2,1.0,0,2.0,0,0,0,0,0,0,0,0,1,1,1,427.6,579.7,471.1,520.0,C,C,C,A,A,A,NaN,A
4573,19,M,1,1,1,2,1,NaN,0,NaN,0,0,0,0,0,0,0,0,1,1,1,639.2,643.8,604.9,640.0,D,F,D,B,B,A,NaN,D
4574,17,M,2,2,3,0,1,NaN,1,NaN,0,0,0,0,0,0,0,0,1,1,1,427.1,467.9,540.2,520.0,C,E,C,A,A,A,NaN,A


Aqui também temos os mesmos casos de dados nulos, logo, como feito no train_clean faremos a substituição de alguns dados pela média arredonda, a substituição dos dados ausentes da questão 27 pelo 'N' e as notas ausentes pela nota 0.

In [155]:
test_clean.isnull().sum().values 

array([   0,    0,    0,    0,    0,    0,    0, 3096,    0, 3096,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1134,
       1134, 1199, 1199,    0,    0,    0,    0,    0,    0, 2488,    0])

In [156]:
test_clean['TP_ENSINO'].mean()

1.1824324324324325

In [0]:
test_clean['TP_ENSINO'].fillna(value=1, inplace=True)

In [158]:
test_clean['TP_DEPENDENCIA_ADM_ESC'].mean()

2.264864864864865

In [0]:
test_clean['TP_DEPENDENCIA_ADM_ESC'].fillna(value=2, inplace=True)

In [0]:
#N é para substituir o NaN proveniente das pessoas que não trabalham, ai não faz sentido elas responderem algo nas questões 27.
test_clean['Q027'].fillna(value='N', inplace=True)

test_clean['NU_NOTA_REDACAO'].fillna(value=0.00, inplace=True)
test_clean['NU_NOTA_CN'].fillna(value=0.00, inplace=True)
test_clean['NU_NOTA_CH'].fillna(value=0.00, inplace=True)
test_clean['NU_NOTA_LC'].fillna(value=0.00, inplace=True)


In [161]:
test_clean.isnull().sum().values 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Por fim, converter os dados inteiros em float.

In [162]:
test_clean.dtypes

NU_IDADE                    int64
TP_SEXO                    object
TP_COR_RACA                 int64
TP_NACIONALIDADE            int64
TP_ST_CONCLUSAO             int64
TP_ANO_CONCLUIU             int64
TP_ESCOLA                   int64
TP_ENSINO                 float64
IN_TREINEIRO                int64
TP_DEPENDENCIA_ADM_ESC    float64
IN_BAIXA_VISAO              int64
IN_CEGUEIRA                 int64
IN_SURDEZ                   int64
IN_DISLEXIA                 int64
IN_DISCALCULIA              int64
IN_SABATISTA                int64
IN_GESTANTE                 int64
IN_IDOSO                    int64
TP_PRESENCA_CN              int64
TP_PRESENCA_CH              int64
TP_PRESENCA_LC              int64
NU_NOTA_CN                float64
NU_NOTA_CH                float64
NU_NOTA_LC                float64
NU_NOTA_REDACAO           float64
Q001                       object
Q002                       object
Q006                       object
Q024                       object
Q025          

In [0]:
test_clean['NU_IDADE'] = test_clean['NU_IDADE'].astype('float64')
test_clean['TP_COR_RACA'] = test_clean['TP_COR_RACA'].astype('float64')
test_clean['TP_NACIONALIDADE'] = test_clean['TP_NACIONALIDADE'].astype('float64')
test_clean['TP_ST_CONCLUSAO'] = test_clean['TP_ST_CONCLUSAO'].astype('float64')
test_clean['TP_ANO_CONCLUIU'] = test_clean['TP_ANO_CONCLUIU'].astype('float64')
test_clean['TP_ESCOLA'] = test_clean['TP_ESCOLA'].astype('float64')
test_clean['IN_TREINEIRO'] = test_clean['IN_TREINEIRO'].astype('float64')
test_clean['IN_BAIXA_VISAO'] = test_clean['IN_BAIXA_VISAO'].astype('float64')
test_clean['IN_CEGUEIRA'] = test_clean['IN_CEGUEIRA'].astype('float64')
test_clean['IN_SURDEZ'] = test_clean['IN_SURDEZ'].astype('float64')
test_clean['IN_DISLEXIA'] = test_clean['IN_DISLEXIA'].astype('float64')
test_clean['IN_DISCALCULIA'] = test_clean['IN_DISCALCULIA'].astype('float64')
test_clean['IN_SABATISTA'] = test_clean['IN_SABATISTA'].astype('float64')
test_clean['IN_GESTANTE'] = test_clean['IN_GESTANTE'].astype('float64')
test_clean['IN_IDOSO'] = test_clean['IN_IDOSO'].astype('float64')
test_clean['TP_PRESENCA_CN'] = test_clean['TP_PRESENCA_CN'].astype('float64')
test_clean['TP_PRESENCA_CH'] = test_clean['TP_PRESENCA_CH'].astype('float64')
test_clean['TP_PRESENCA_LC'] = test_clean['TP_PRESENCA_LC'].astype('float64')

In [164]:
test_clean.dtypes

NU_IDADE                  float64
TP_SEXO                    object
TP_COR_RACA               float64
TP_NACIONALIDADE          float64
TP_ST_CONCLUSAO           float64
TP_ANO_CONCLUIU           float64
TP_ESCOLA                 float64
TP_ENSINO                 float64
IN_TREINEIRO              float64
TP_DEPENDENCIA_ADM_ESC    float64
IN_BAIXA_VISAO            float64
IN_CEGUEIRA               float64
IN_SURDEZ                 float64
IN_DISLEXIA               float64
IN_DISCALCULIA            float64
IN_SABATISTA              float64
IN_GESTANTE               float64
IN_IDOSO                  float64
TP_PRESENCA_CN            float64
TP_PRESENCA_CH            float64
TP_PRESENCA_LC            float64
NU_NOTA_CN                float64
NU_NOTA_CH                float64
NU_NOTA_LC                float64
NU_NOTA_REDACAO           float64
Q001                       object
Q002                       object
Q006                       object
Q024                       object
Q025          

Essa tabela não contem as notas (o desafio é prevê-las), logo bastamos preparar a variável dos previsores

In [165]:
# colocando todas as colunas exceto NU_NOTA_MT
predictors_answer = test_clean.values
predictors_answer

array([[22.0, 'F', 3.0, ..., 'C', 'C', 'A'],
       [26.0, 'F', 3.0, ..., 'B', 'F', 'A'],
       [21.0, 'M', 1.0, ..., 'A', 'N', 'A'],
       ...,
       [19.0, 'M', 1.0, ..., 'A', 'N', 'D'],
       [17.0, 'M', 2.0, ..., 'A', 'N', 'A'],
       [30.0, 'M', 1.0, ..., 'B', 'B', 'A']], dtype=object)

Como feito no caso da train, faremos o mesmo trabalho com os dados categóricos para trabalhar com eles de forma numérica.

In [0]:
labelenconder = LabelEncoder()
predictors_answer[:, 1] = labelenconder.fit_transform(predictors_answer[:, 1])
predictors_answer[:, -8] = labelenconder.fit_transform(predictors_answer[:, -8])
predictors_answer[:, -7] = labelenconder.fit_transform(predictors_answer[:, -7])
predictors_answer[:, -6] = labelenconder.fit_transform(predictors_answer[:, -6])
predictors_answer[:, -5] = labelenconder.fit_transform(predictors_answer[:, -5])
predictors_answer[:, -4] = labelenconder.fit_transform(predictors_answer[:, -4])
predictors_answer[:, -3] = labelenconder.fit_transform(predictors_answer[:, -3])
predictors_answer[:, -2] = labelenconder.fit_transform(predictors_answer[:, -2])
predictors_answer[:, -1] = labelenconder.fit_transform(predictors_answer[:, -1])

#onehotenconder = OneHotEncoder(categories = [1,-24,-25,-26,-27,-28,-29,-30,-31])
#previsores = onehotenconder.fit_transform(previsores).toarray()

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1,-1,-2,-3,-4,-5,-6,-7,-8])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)

predictors_answer = ct.fit_transform(predictors_answer)

In [167]:
predictors_answer

array([[1.0, 0.0, 1.0, ..., 443.5, 431.8, 420.0],
       [1.0, 0.0, 1.0, ..., 491.1, 548.0, 580.0],
       [0.0, 1.0, 1.0, ..., 622.7, 613.6, 320.0],
       ...,
       [0.0, 1.0, 0.0, ..., 643.8, 604.9, 640.0],
       [0.0, 1.0, 1.0, ..., 467.9, 540.2, 520.0],
       [0.0, 1.0, 1.0, ..., 0.0, 0.0, 0.0]], dtype=object)

Aqui aplicamos a rede neural *regressor* na variável dos previsores.

In [0]:
result_answer = regressor.predict(predictors_answer)

In [169]:
#previsoes
result_answer

array([[ 4.1972998e+02],
       [ 4.5308002e+02],
       [ 5.8609253e+02],
       ...,
       [ 7.2026746e+02],
       [ 4.6905289e+02],
       [-5.5717230e-02]], dtype=float32)

# Fazendo o dataframe

Aqui apenas monto o dataframe com a coluna *'NU_INSCRICAO'* e *'NU_NOTA_MT'* como solicitado pelo desafio.

In [0]:
subscribes = []
for item in test['NU_INSCRICAO']:
  subscribes.append(item)

grades_answer = []
for item in range(0,4576,1):
  if result_answer[item][0] < 0:
    grades_answer.append(0.00)
  else:
    grades_answer.append(result_answer[item][0])

data = np.array([subscribes, grades_answer])

In [0]:
data = np.transpose(data)


In [174]:
delievery = pd.DataFrame(data=data[:][:], columns=['NU_INSCRICAO', 'NU_NOTA_MT'])
delievery

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,419.72998
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,453.08002
2,b38a03232f43b11c9d0788abaf060f7366053b6d,586.0925
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.0
4,715494628a50142ce8cb17191cfe6d0f3cae0934,580.5191
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,451.52603
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,459.46536
4573,655fa6306720ff16e825903b5422a46608a77545,720.26746
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,469.0529


In [0]:
delievery = delievery.set_index('NU_INSCRICAO') #retirando a coluna index

In [0]:
delievery.to_csv('answers.csv')
files.download('answers.csv')